## Run in blender

In [6]:
import bpy
import glob
local_path = "/Users/pataranansethpakdee/Documents/GitHub/"
vrm_filepath = local_path+"3D-Waifu-Model-Generator/Datasets/3D_VRMModel/"
processed_filepath = local_path + "3D-Waifu-Model-Generator/Datasets/3D_ProcessedModel/"
image_filepath = local_path + "3D-Waifu-Model-Generator/Datasets/2D_Image/"
vrm_files = glob.glob(f'{vrm_filepath}*')

def purge_orphans():
    bpy.ops.outliner.orphans_purge(do_local_ids=True, do_linked_ids=True, do_recursive=True)

def clean_scene():
    scene = bpy.context.scene
    bpy.data.scenes.new("Scene")
    bpy.data.scenes.remove(scene, do_unlink=True)
    purge_orphans()
    
def set_new_camera():
    scene = bpy.context.scene
    cam_data = bpy.data.cameras.new(name="Camera")
    cam = bpy.data.objects.new(name="Camera", object_data=cam_data)
    scene.collection.objects.link(cam)
    scene.camera = cam
    cam.location = (0, -5.3, 0.8)
    cam.rotation_euler = (1.5708, 0, 0) 

def new_scene():
    clean_scene()
    set_new_camera()

count = 0

for file in vrm_files:
    
    file_name = str(count).zfill(4)

    clean_scene()
    set_new_camera()
    bpy.ops.import_scene.vrm(filepath=file)
    scene = bpy.context.scene


    # Render settings
    scene.render.filepath = f'{image_filepath}{file_name}.png'
    scene.render.image_settings.file_format = 'PNG'
    scene.render.resolution_x = 1920
    scene.render.resolution_y = 1080

    # Render the image
    bpy.ops.render.render(write_still=True)

    #bpy.ops.export_scene.gltf(filepath=f"{processed_filepath}{file_name}.glb")
    bpy.ops.export_scene.gltf(filepath=f"{processed_filepath}{file_name}.glb", 
                         export_format='GLB', 
                         export_image_format='JPEG',
                         export_image_add_webp=True,
                         export_image_webp_fallback=True,
                         export_texcoords=True,
                         export_normals=True,
                         export_materials='EXPORT',
                         export_vertex_color='MATERIAL',
                         export_all_vertex_colors=True)
    count+=1



# Preprocessing

Run this in Window 

In [ ]:
#Convert .ply to point cloud
import trimesh
import glob , os
ply_path = "Datasets/3D_ProcessedModel"
ply_files = glob.glob(r"Datasets/3D_ProcessedModel/*")

for file in ply_files:
    file_name = (file.split('/')[-1]).split(".")[0]
    path = f"{ply_path}/{file_name}.glb"
    scene = trimesh.load(file)

    # Traverse all geometries (meshes) in the scene
    point_clouds = []
    for name, mesh in scene.geometry.items():
        vertices = mesh.vertices
        colors = mesh.visual.to_color().vertex_colors  # Optional: may be texture-based
        
        # Sample points from the mesh
        num_samples = 5000  # Adjust this number for more points
        if len(vertices) > num_samples:
            indices = np.random.choice(len(vertices), num_samples, replace=False)
            sampled_vertices = vertices[indices]
            sampled_colors = colors[indices]
        else:
            sampled_vertices = vertices
            sampled_colors = colors

        # Combine vertices and colors into a point cloud
        point_cloud_with_color = [(*v, *c[:3]) for v, c in zip(sampled_vertices, sampled_colors)]
        point_clouds.append(point_cloud_with_color)
    all_points = [point for pc in point_clouds for point in pc]
    points = np.array([p[:3] for p in all_points])
    colors = np.array([p[3:] for p in all_points]) / 255.0  # Normalize color values

    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    o3d.io.write_point_cloud(f"Processed_Data/3D_PointCloud/{file_name}.ply", pcd)
#o3d.io.write_point_cloud(f"Processed_Data/3D_PointCloud/{file_name}.pts",pcd)


In [5]:
import torch
import numpy as np
from plyfile import PlyData
def load_pointcloud(ply_file):
    ply_data = PlyData.read(ply_file)
    vertices = np.vstack([ply_data['vertex'].data['x'], 
                          ply_data['vertex'].data['y'], 
                          ply_data['vertex'].data['z']]).T
    colors = np.vstack([ply_data['vertex'].data['red'], 
                        ply_data['vertex'].data['green'], 
                        ply_data['vertex'].data['blue']]).T
    return vertices, colors

(array([[-5.02770916e-02,  9.37048197e-01,  6.82605505e-02],
        [-7.08752945e-02,  9.46168542e-01,  8.04238841e-02],
        [-2.99292710e-02,  1.61273026e+00, -6.11106046e-02],
        ...,
        [ 9.65739861e-02,  1.51713276e+00,  1.42953813e-03],
        [-1.84429940e-02,  1.53768718e+00,  7.59180263e-02],
        [ 4.21817377e-02,  1.60382962e+00,  2.55565792e-02]]),
 array([[ 51,  48,  51],
        [ 54,  54,  52],
        [248, 216, 190],
        ...,
        [165, 130, 133],
        [210, 193, 199],
        [228, 210, 211]], dtype=uint8))